In [1]:
using Plots;
pythonplot();
using LinearAlgebra;
using JLD2;
using LaTeXStrings;

include("../../Hamiltonians/H2.jl");
using .H2;
include("../../Hamiltonians/H4.jl");
using .H4;
include("../../Helpers/ChaosIndicators.jl");
using .ChaosIndicators;

colors = [:dodgerblue :darkviolet :limegreen :indianred :magenta :darkblue :aqua :deeppink :dimgray :red :royalblue :slategray :black :lightseagreen :forestgreen :palevioletred]
markers = ["o","x","v","*","H","D","s","d","P","2","|","<",">","_","+",","]
markers_line = ["-o","-x","-v","-H","-D","-s","-d","-P","-2","-|","-<","->","-_","-+","-,"]


15-element Vector{String}:
 "-o"
 "-x"
 "-v"
 "-H"
 "-D"
 "-s"
 "-d"
 "-P"
 "-2"
 "-|"
 "-<"
 "->"
 "-_"
 "-+"
 "-,"

Level Spacing Ratio:

In [2]:
function GetLevelSpacingRatioData(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)
    r = Vector{Vector{Float64}}(undef, length(L′s))

    for (i,L) in enumerate(L′s)
        println(L, "--------")
        r[i] = ChaosIndicators.LevelSpacingRatio(L, q′s, maxNumbOfIter)

        folder = jldopen("./Data-ChaosIndicators/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter).jld2", "w");
        folder["r"] = r[i];
        folder["q′s"] = q′s;
        close(folder)        
    end

    println(r)

end


GetLevelSpacingRatioData (generic function with 1 method)

In [3]:
function PlotLevelSpacingRatio(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64, dataLocation_prefix:: String, title:: String)

    levelSpacingRatio = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft, title = "Level spacing ratio");

    plot!(levelSpacingRatio, q′s, 0.3863 .* ones(Float64, length(q′s)), ls=:dash, label="Poisson");
    plot!(levelSpacingRatio, q′s, 0.5359 .* ones(Float64, length(q′s)), ls=:dash, label="GOE");

    for (i,L) in enumerate(L′s) 
        folder = jldopen(dataLocation_prefix * "_L$(L)_Iter$(maxNumbOfIter).jld2", "r");
        r = folder["r"];
        q′s = folder["q′s"];
        close(folder);

        plot!(levelSpacingRatio, q′s, r, lc = colors[i], label="L=$(L)");

    end

    xlabel!(L"q");
    ylabel!(L"r");

    return levelSpacingRatio;
end

PlotLevelSpacingRatio (generic function with 1 method)

Informational entropy

In [4]:
function GetInformationalEntropyData(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64)
    S = Vector{Vector{Float64}}(undef, length(L′s))

    for (i,L) in enumerate(L′s)
        println(L, "--------")
        S[i] = ChaosIndicators.InformationalEntropy(L, q′s, maxNumbOfIter)

        folder = jldopen("./Data-ChaosIndicators/InformationalEntropy_L$(L)_Iter$(maxNumbOfIter)_fixedNormalization.jld2", "w");
        folder["S"] = S[i];
        folder["q′s"] = q′s;
        close(folder)        
    end

    println(S)

end

GetInformationalEntropyData (generic function with 1 method)

In [5]:
function PlotInformationalEntropy(L′s:: Vector{Int}, q′s:: Vector{Float64}, maxNumbOfIter:: Int64, dataLocation_prefix:: String, title:: String)
    informationalEntropy = plot(xscale=:log10, minorgrid=true, framestyle = :box, legend=:topleft, title = title);

    plot!(informationalEntropy, q′s,  ones(Float64, length(q′s)), ls=:dash, label="");

    for (i,L) in enumerate(L′s) 
        folder = jldopen(dataLocation_prefix * "_L$(L)_Iter$(maxNumbOfIter).jld2", "r");
        S = folder["S"];
        q′s = folder["q′s"];
        close(folder);
        
        plot!(informationalEntropy, q′s, S, lc = colors[i], label="L=$(L)");

    end

    xlabel!(L"q");
    ylabel!(L"S");

    return informationalEntropy;
end

PlotInformationalEntropy (generic function with 1 method)

Retrieve data:

In [30]:
L′s = [6,8,10,12];
x= LinRange(-5, 1, 20);
q′s = 10 .^(x);
maxNumbOfIter = 100

# GetLevelSpacingRatioData(L′s, q′s, maxNumbOfIter)

100

In [31]:
L′s = [6,8,10,12];
x= LinRange(-5, 1, 20);
q′s = 10 .^(x);
maxNumbOfIter = 200

# GetInformationalEntropyData(L′s, q′s, maxNumbOfIter)

200

In [32]:
function JoinTwoLSRFiles(L′s, maxNumbOfIter1, maxNumbOfIter2)
    
    for L in L′s
        folder1 = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter1).jld2", "r");
        r1 = folder1["r"];
        q′s1 = folder1["q′s"];
        close(folder1);
        
        folder2 = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter2).jld2", "r");
        r2 = folder2["r"];
        q′s2 = folder2["q′s"];
        close(folder2);

        q′s1 == q′s2 ? println("q-ji so ok") : println("q-ji niso ok");

        maxNumbOfIter = maxNumbOfIter1 + maxNumbOfIter2;
        r = (r1*maxNumbOfIter1 + r2*maxNumbOfIter2)/maxNumbOfIter;

        folder = jldopen("./Data/LevelSpacingRatio_L$(L)_Iter$(maxNumbOfIter).jld2", "w");
        folder["r"] = r;
        folder["q′s"] = q′s;
        close(folder);
    end
end

# L′s = [6, 8, 10, 12];
# maxNumbOfIter1 = 400;
# maxNumbOfIter2 = 100;

# JoinTwoLSRFiles(L′s, maxNumbOfIter1, maxNumbOfIter2)

JoinTwoLSRFiles (generic function with 1 method)

In [6]:
function Plot1(L′s_lsr:: Vector{Int}, L′s_ie:: Vector{Int}, q′s_lsr:: Vector{Float64}, q′s_ie:: Vector{Float64}, maxNumbOfIter_lsr:: Int64, maxNumbOfIter_ie:: Int64)
    dataLocation_prefix1:: String = "./Data-ChaosIndicators/LevelSpacingRatio";
    dataLocation_prefix2:: String = "./Data-ChaosIndicators/InformationalEntropy";

    levelSpacingRatio = PlotLevelSpacingRatio(L′s_lsr, q′s_lsr, maxNumbOfIter_lsr, dataLocation_prefix1, "Level Spacing Ratio");
    informationalEntropy = PlotInformationalEntropy(L′s_ie, q′s_ie, maxNumbOfIter_ie, dataLocation_prefix2, "Informational Entropy");

    ratio = 2.3;
    dim = 500;
    p = plot(levelSpacingRatio, informationalEntropy, layout=(1,2), size=(Int(dim*ratio),dim));

    savefig(p, "./Images-ChaosIndicators/ChaosIndicators_L$(L′s_lsr)_$(maxNumbOfIter_lsr)_$(maxNumbOfIter_lsr).pdf");
end

Plot1 (generic function with 1 method)

In [34]:
L′s_lsr = [6,8,10,12];
maxNumbOfIter_lsr = 500;
x_lsr= LinRange(-5, 1, 10);
q′s_lsr = 10 .^(x);

L′s_ie  = [8,10,12];
maxNumbOfIter_ie = 200;
x_ie= LinRange(-5, 1, 10);
q′s_ie = 10 .^(x);

Plot1(L′s_lsr, L′s_ie, q′s_lsr, q′s_ie, maxNumbOfIter_lsr, maxNumbOfIter_ie)

"/Users/rokpintar/Desktop/FAX/MAG 2. /Magisterska naloga/Magisterska-Naloga-Program/MastersDegree/Plotting/ChaosIndicators/Images-ChaosIndicators/ChaosIndicators_L[6, 8, 10, 12]_500_500.pdf"